In [27]:
from qiskit.circuit.library import NLocal
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit import ParameterVector
from random import uniform
from math import pi

from pennylane import numpy as np

import pandas as pd

import qutip

n_wires = 6     #Como precisamos fornecer o tamanho dos blocos de rotação e emaranhamento, para mudar n_wires teremos que mudar também o tamanho dos blocos 
n_layers = 1
iteracoes = 5
n_targets = 3

def NLocal2(n_layers, n_wires, n_targets):


    # rotation block:
    rot = QuantumCircuit(2)
    params = ParameterVector('r', 2)
    params = (uniform(0,2*pi), uniform(0,2*pi))
    print("\nParâmetros de rotação: {}".format(params))
    rot.ry(params[0], 0)
    rot.rz(params[1], 1)

    # entanglement block:
    ent = QuantumCircuit(4)
    params = ParameterVector('e', 3)
    params = (uniform(0,2*pi), uniform(0,2*pi), uniform(0,2*pi))
    print("Parâmetros de rotação/emaranhamento: {}".format(params))
    ent.crx(params[0], 0, 1)
    ent.crx(params[1], 1, 2)
    ent.crx(params[2], 2, 3)


    #Cria dados
    def normaliza(v):
        v = v / (np.linalg.norm(v))    
        return v

    def cria_alvos(n_targets, n_wires):
        target_states = []
        for i in range(n_targets):
            target_states.append(normaliza(np.random.rand(1, 2**(n_wires))))
        return target_states
    
    qc_nlocal = NLocal(num_qubits=n_wires, rotation_blocks=rot,
                   entanglement_blocks=ent, reps=n_layers, entanglement='linear',
                   skip_final_rotation_layer=True, insert_barriers=True)
        
    print("\n")
    print("Circuito gerado: {}".format(qc_nlocal.decompose().draw()))

    backend = Aer.get_backend('statevector_simulator')
    ket = execute(qc_nlocal, backend).result().get_statevector()

    target_states = cria_alvos(n_targets, n_wires)

    ket = np.array(ket)    #Transformamos nosso StateVector em um tensor do numpy (do pennylane)
    

    def ket_in_amplt(v):
        aux = []
        for i in range(len(v)):
            v[i] = np.dot(v[i],v[i])
            v[i] = np.linalg.norm(v[i])
            aux.append(float(v[i].real))
        aux = np.array(aux)    
        return aux
    amplt = ket_in_amplt(ket)


    print("\n")
    #print("Estado gerado pelo modelo: {}".format(ket))



    def fidelity(target_states, amplt):
        fidelity_sum = 0
        for target_state in target_states:
            output_probs = amplt
            fidelity = np.vdot(target_state, output_probs) ** 2
            fidelity_sum += fidelity
    
        average_fidelity = fidelity_sum / len(target_states)
        return average_fidelity
    

    def compute_Q_ptrace(ket, N):
        ket = qutip.Qobj(ket, dims=[[2]*(N), [1]*(N)]).unit()
        entanglement_sum = 0
        for k in range(N):
            rho_k_sq = ket.ptrace([k])**2
            entanglement_sum += rho_k_sq.tr()  
   
        Q = 2*(1 - (1/N)*entanglement_sum)
        return Q




    Q = compute_Q_ptrace(ket, n_wires)
    print("\nGrau de Emaranhamento: {}".format(Q))

    E = fidelity(target_states, amplt)
    print("Expressabilidade: {}".format(E))

    #print("\n\nEstado gerado: {}".format(ket))
    #print("\n\n\n\n\n")

    k=[Q,E]
    return k



k=[]
for i in range(iteracoes):
    print("\n=============")
    print("= Modelo: {} =".format(i+1))
    print("=============\n")
    k.append(NLocal2(n_layers, n_wires, n_targets))


print("\n\nResultados:\n")
for i in range(iteracoes):
    print(" Modelo {} \n Grau de emaranhamento: {} \n Expressabilidade: {} \n".format(i+1,k[i][0],k[i][1]))


def pand(k):
    eman=[]
    exp=[]

    for i in range(iteracoes):
        eman.append(k[i][0])
    for i in range(iteracoes):
        exp.append(k[i][1]) 

    dic = {"Grau de Emaranhamento": eman, "Expressabilidade":exp}
    df = pd.DataFrame(dic)
    return df

df = pand(k)
df.describe()



= Modelo: 1 =


Parâmetros de rotação: (2.943106907411095, 3.506341961956366)
Parâmetros de rotação/emaranhamento: (5.54897899581824, 3.888033580364122, 0.8717760245555114)


Circuito gerado:      ┌────────────┐ ░                                                       »
q_0: ┤ Ry(2.9431) ├─░───────■───────────────────────────────────────────────»
     ├────────────┤ ░ ┌─────┴─────┐                                         »
q_1: ┤ Rz(3.5063) ├─░─┤ Rx(5.549) ├──────■───────────────────────────■──────»
     ├────────────┤ ░ └───────────┘┌─────┴─────┐               ┌─────┴─────┐»
q_2: ┤ Ry(2.9431) ├─░──────────────┤ Rx(3.888) ├───────■───────┤ Rx(5.549) ├»
     ├────────────┤ ░              └───────────┘┌──────┴──────┐└───────────┘»
q_3: ┤ Rz(3.5063) ├─░───────────────────────────┤ Rx(0.87178) ├─────────────»
     ├────────────┤ ░                           └─────────────┘             »
q_4: ┤ Ry(2.9431) ├─░───────────────────────────────────────────────────────»
     ├────────────┤ ░      

,Grau de Emaranhamento,Expressabilidade
count,5.000000,5.000000
mean,0.178016,0.011936
std,0.240875,0.003080
min,0.002281,0.007742
25%,0.034856,0.009926
50%,0.081713,0.012585
75%,0.179172,0.014239
max,0.592058,0.015190
